# Qual o melhor meio de representar esso texto corrido para análise de dados?

In [11]:
import pandas as pd
import numpy as np

import re

In [4]:
df = pd.read_csv('https://raw.githubusercontent.com/leoferro/analise_textual_capivari/main/raw_data.csv')

In [5]:
df.head()

,Protocolo,Data,Assunto,Situação,Regime,Autoria
0,00594/2024,20/02/2024,Solicita informações sobre nebulização de inse...,Protocolada,Ordinário,MARCOS SABINO FERNANDES
1,00593/2024,20/02/2024,Moção de Repúdio ao Excelentíssimo Presidente ...,Protocolada,Ordinário,JOSÉ EDUARDO DE CAMPOS PACHECO
2,00580/2024,19/02/2024,Solicita melhorias nas ruas do bairro Cancian,Protocolada,Ordinário,THIAGO JUNIOR ANESIO BRAGGION
3,00533/2024,19/02/2024,Solicita troca de lâmpadas queimadas na rodovi...,Protocolada,Ordinário,DENILTON ROCHA DOS SANTOS
4,00526/2024,19/02/2024,Solicita melhorias na ponte (Pontinha do SAAE)...,Protocolada,Ordinário,RUAN HENRIQUE PELLEGRINI BRAGGION


In [31]:
def transform(data):
  data = (
      data
       .str.lower()
       .str.strip()
       .str.replace(r'[()/-/{/}//]','',regex=True)
       .str.replace(r'[á`aãâ]','a'    ,regex=True)
       .str.replace(r'[éê]','e'       ,regex=True)
       .str.replace(r'[í]','i'        ,regex=True)
       .str.replace(r'[óõ]','o'       ,regex=True)
       .str.replace(r'[ú]','u'        ,regex=True)
  )

  return data

transform(df.Assunto)

0       solicita informaçoes sobre nebulizaçao de inse...
1       moçao de repudio ao excelentissimo presidente ...
2           solicita melhorias nas ruas do bairro cancian
3       solicita troca de lampadas queimadas na rodovi...
4       solicita melhorias na ponte pontinha do saae e...
                              ...                        
1495    manutençao e higienizaçao do bebedouro localiz...
1496    conserto no inicio da avenida josefina giovana...
1497    cascalho ou raspas de asfalto na rua arthur no...
1498    notifique a cpfl companhia paulista de força e...
1499    cimentaçao no espaço onde esta instalado o par...
Name: Assunto, Length: 1500, dtype: object